# Aircraft Registration Prefix (ARP) Playground

This notebook holds helping funcitons to help the ETL process of creating the ARP dataset

In [1]:
import re

#Flight Num 1
l = ["VY7893", "VY2110", "W63651"]
for i in range(len(l)):
    result = re.match("^[A-Z]{2}\d{3,4}$", l[i])
print(result)

None


In [ ]:
#PROCESS ITUS Helping Functions

import csv

def load_itus():
    itus = []
    with open('itu-prefix.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                line_count += 1
            #print(row)
            itus.append(row)
            line_count += 1
        print(f'Processed {line_count} lines.')
        return itus

itus = load_itus()

#def extract_air
def convert_itus(itus):
    
    def add(c, x, m):
        #print("C:%s, M:%s" % (c,m))
        if c < m:
            return chr(ord(c)+x)
    
    def slap(l):
        return ''.join(str(x) for x in l)
    
    def gen_offset_list(start, end):
        #print("Start=%s, End=%s" % (start,end))
        result = []
    
        c_start = start
        ls = list(c_start)
        le = list(end)
        index = len(ls)-1 
        
        if len(ls) == 2:
        
            result.append(c_start)
            for i in reversed(range(len(ls))):
                ls = list(c_start)
                while ls[i] is not le[i]:
                    ls[i] = add(ls[i], 1, le[i])
                    result.append(slap(ls))    
            #print("%s==%s" % (ls[i],le[i]))
            #result.append(end)
        
        else:
            #print("Tags are not the same size..")
            result.append("%s-%s" % (start,end))
        return result
       
    new_list = {}
    
    for i in range(len(itus)):
        #print(itus[i])
        #print(itus[i]["nation"])
        #print(itus[i]["callsign"])
        nation = itus[i]["nation"]
        c_list = itus[i]["callsign"].split(",")
        r_list = []
        #print("Nation:%s, Old List %s" % (nation, c_list))
        for ele in c_list:
            if "-" in ele:
                #print(ele)
                s = ele.strip().split("-")
                r_list += gen_offset_list(s[0], s[1])
            else:
                r_list.append(ele)
        #print("Nation:%s, New List: %s " % (nation, r_list))
        new_list[nation] = r_list
    
    return new_list

result_list = convert_itus(itus)
#print(result_list)

# with open('processed_itu.csv', 'w') as csv_file:
#     writer = csv.writer(csv_file)
#     for key, value in result_list.items():
#         writer.writerow([key, value])

In [ ]:
# import pickle

# x = []
# with open('processed_itu.csv','r') as f:
#     reader = csv.reader(f)
#     for line in reader: x.append(line)

# with open("itu.pickle",'wb') as f:
#     pickle.dump(x, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
#Load ITUS

# def load_itus(path):
#     with open(path,'rb') as f:
#         x = pickle.load(f)
#         return x

# itus = load_itus("itu.pickle")
# print(len(itus))

In [4]:
#Load the processed ITU dataset with nation,description,iso codes, callsign, suffix

import pandas as pd
import ast

df = pd.read_csv("processed_itu_countries.csv")
df.head(60)

,nation,description,iso codes,callsign,suffix
0,Afghanistan,general,"['AF', 'AFG']","['T6', 'YA']",['AAA-ZZZ']
1,Albania,general,"['AL', 'ALB']",['ZA'],['AAA-ZZZ']
2,Algeria,civilian,"['AL', 'ALB']","['7R', '7T', '7U', '7V', '7W', '7X', '7Y']",['VAA-VZZ']
3,Algeria,military,"['DZ', 'DZA']","['7R', '7T', '7U', '7V', '7W', '7X', '7Y']",['WAA-WZZ']
4,Andorra,general,"['AD', 'AND']",['C3'],['AAA-ZZZ']
5,Angola,general,"['AO', 'AGO']","['D2', 'D3']",['AAA-ZZZ']
6,Anguilla,general,"['AI', 'AIA']",['VP-A'],['AA-ZZ']
7,Antigua and Barbuda,general,"['AG', 'ATG']",['V2'],['AAA-ZZZ']
8,Argentina,general,"['AR', 'ARG']","['AY', 'AZ', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7...",['AAA-ZZZ']
9,Argentina,military,"['AR', 'ARG']",['LQ'],['AAA-ZZZ']


In [5]:
#Regex Playground
import re
l = ["VR-7", "VY2110", "W63651", "N199ZZ"]

exp = "(AA|AB|N)-{0,1}(([1-9][0-9]{0,2}[A-Z]{2}|[1-9][0-9]{0,3}[A-Z]|[1-9][0-9]{0,4})|([A-Z0-9]{1,4}))$"
#exp = "([A-Z0-9]{1,3})-{0,1}([A-Z0-9]{1,4})"

# Max length of suffix is 5
# Max length of designator is 3 (without dash)
# Sometimes dash exists
# Othertimes it doesn't

for i in range(len(l)):
    #no dash
    result_dash = re.match(exp, l[i])
    print(result_dash)

None
None
None
<_sre.SRE_Match object; span=(0, 6), match='N199ZZ'>


In [6]:
#Create Expressions from dataframe
import ast

def generate_expressions(description, prefixs, suffixs):

    # prefix: country identifier
    # suffix: expression

    
    manual_tag = "MANUAL"
    exp = "\""
    end = "$"
    # US - "(AA|AB|N)-{0,1}([1-9][0-9]{0,2}[A-Z]{2}|[1-9][0-9]{0,3}[A-Z]|[1-9][0-9]{0,4})$"
    
    # Append Prefix
    exp = "("
    count = 0
    for prefix in prefixs:
        if count < len(prefixs)-1:
            exp = "%s%s|" % (exp,prefix.strip())
        else:
            exp = "%s%s)" % (exp,prefix.strip())
        count += 1
    
    #Append dash (If a general... append additional dash)
    exp = "%s(-{0,1}(" % exp if description.strip() == 'general' else "%s(-{0,1}" % exp
    
    #Append Suffixes
    exp = "%s(" % exp
    
    count = 0
    for suffix in suffixs:
        
        s_list = suffix.strip().split("-")

        #Append ruleset
        if len(s_list) == 1:

            if '*' in suffix:
                #print("* in suffix, suffix=%s" % (suffix))
                #generate token based rules
                t1 = list(s_list[0])
                for c in range(len(t1)):
                    if t1[c] == '*':
                        #generate rule
                        exp = "%s[A-Z0-9]|" % (exp) if count < len(suffixs)-1 else "%s[A-Z0-9]" % (exp)
                    else:
                        #normal rule
                        exp = "%s%s|" % (exp, t1[c]) if count < len(suffixs)-1 else "%s%s" % (exp, t1[c])
            else:
                # s_list must be a rule... unless it's a star!
                exp = "%s%s|" % (exp, suffix) if count < len(suffixs)-1 else  "%s%s" % (exp, suffix)
            
        elif len(s_list) == 2:
            # s_list must have a range.. break it down
            if len(s_list[0]) == len(s_list[1]):
                #create the ruleset...
                t1 = list(s_list[0])
                t2 = list(s_list[1])
                r_suffix = ""
                for c in range(len(t1)):
                    #if they are the same
                    if t1[c] == t2[c] and t1[c] is not '*':
                        r_suffix = "%s%s" % (r_suffix, t1[c]) 
                    elif t1[c] == t2[c] and t1[c] is '*':
                        r_suffix = "%s[A-Z0-9]" % (r_suffix)
                    else:
                        #range
                        r_suffix = "%s[%s-%s]" % (r_suffix, t1[c], t2[c])
                        
                exp = "%s%s|" % (exp, r_suffix) if count < len(suffixs)-1 else  "%s%s" % (exp, r_suffix)
                
            else:
                #This needs to be done manually
                #tag with manual tag
                #print("SPECIAL CASE : %s" % suffix)
                
                #specific cases
                s_case = manual_tag

                if suffix == "AZ1-AZ999":
                    s_case = "[A-Z]{2}[1-9][0-9]{0,2}"
                elif suffix == '01-499':
                    s_case = "[0-4][1-9][0-9]|[0-4][1-9]"
                elif suffix == "81-8999":
                    s_case = "8[1-9][0-9]{0,2}"
                elif suffix == "91-9999":
                    s_case = "9[1-9][0-9]{0,2}"
                elif suffix == "UA-001-UA-999":
                    s_case = "UA-[0-9][0-9][1-9]"
                elif suffix == '100-9999':
                    s_case = "[1-9][0-9][0-9]{1,2}"  
                elif suffix == 'AA1-ZZ99':
                    s_case = "[A-Z]{2}[1-9][0-9]{0,1}"
                elif suffix == 'AAA1-ZZZ99':
                    s_case = "[A-Z]{3}[1-9][0-9]{0,1}"
                elif suffix == 'AAA-9999':
                    s_case = "[A-Z0-9]{3}[0-9]{0,1}"
                elif suffix == '1-99999':
                    s_case = "[1-9][0-9]{0,4}"
                elif suffix == '1A-9999Z':
                    s_case = "[1-9][0-9]{0,3}[A-Z]"
                elif suffix == '1AA-999ZZ':
                    s_case = "[1-9][0-9]{0,2}[A-Z]{2}"
                elif suffix == 'AA1-ZZ99':
                    s_case = "[A-Z]{2}[1-9][0-9]{0,1}"
                else:
                    s_case = manual_tag
                    
                exp = "%s%s|" % (exp, s_case) if count < len(suffixs)-1 else "%s%s" % (exp, s_case)
                 
        else:
            #Manual Suffix 
            exp = "%s%s|" % (exp, manual_tag) if count < len(suffixs)-1 else "%s%s" % (exp, manual_tag)

        count += 1
        
    exp = "%s)|([A-Z0-9]{1,4}))){0,1}$" % (exp) if description.strip() == 'general' else "%s)){0,1}$" % exp
    
    #print("prefix=%s, suffix=%s exp=\"%s\"" % (prefixs, suffixs, exp))
    
    return exp

    
res = []
for index, entry in df.iterrows():
    res.append(generate_expressions(entry['description'], ast.literal_eval(entry['callsign']), ast.literal_eval(entry['suffix'])))
    
print(len(res))

408


In [7]:
#Append To Dataframe
df['regex'] = res
df.head(30)


#Save Dataframe To New CSV
df.to_csv('processed_itu_countries_regex.csv', index=False)

In [8]:
df.head()

,nation,description,iso codes,callsign,suffix,regex
0,Afghanistan,general,"['AF', 'AFG']","['T6', 'YA']",['AAA-ZZZ'],"(T6|YA)(-{0,1}(([A-Z][A-Z][A-Z])|([A-Z0-9]{1,4..."
1,Albania,general,"['AL', 'ALB']",['ZA'],['AAA-ZZZ'],"(ZA)(-{0,1}(([A-Z][A-Z][A-Z])|([A-Z0-9]{1,4}))..."
2,Algeria,civilian,"['AL', 'ALB']","['7R', '7T', '7U', '7V', '7W', '7X', '7Y']",['VAA-VZZ'],"(7R|7T|7U|7V|7W|7X|7Y)(-{0,1}(V[A-Z][A-Z])){0,1}$"
3,Algeria,military,"['DZ', 'DZA']","['7R', '7T', '7U', '7V', '7W', '7X', '7Y']",['WAA-WZZ'],"(7R|7T|7U|7V|7W|7X|7Y)(-{0,1}(W[A-Z][A-Z])){0,1}$"
4,Andorra,general,"['AD', 'AND']",['C3'],['AAA-ZZZ'],"(C3)(-{0,1}(([A-Z][A-Z][A-Z])|([A-Z0-9]{1,4}))..."


In [9]:
#Read new dataframe in (with regex's)
df = pd.read_csv("processed_itu_countries_regex.csv")
df.head()

,nation,description,iso codes,callsign,suffix,regex
0,Afghanistan,general,"['AF', 'AFG']","['T6', 'YA']",['AAA-ZZZ'],"(T6|YA)(-{0,1}(([A-Z][A-Z][A-Z])|([A-Z0-9]{1,4..."
1,Albania,general,"['AL', 'ALB']",['ZA'],['AAA-ZZZ'],"(ZA)(-{0,1}(([A-Z][A-Z][A-Z])|([A-Z0-9]{1,4}))..."
2,Algeria,civilian,"['AL', 'ALB']","['7R', '7T', '7U', '7V', '7W', '7X', '7Y']",['VAA-VZZ'],"(7R|7T|7U|7V|7W|7X|7Y)(-{0,1}(V[A-Z][A-Z])){0,1}$"
3,Algeria,military,"['DZ', 'DZA']","['7R', '7T', '7U', '7V', '7W', '7X', '7Y']",['WAA-WZZ'],"(7R|7T|7U|7V|7W|7X|7Y)(-{0,1}(W[A-Z][A-Z])){0,1}$"
4,Andorra,general,"['AD', 'AND']",['C3'],['AAA-ZZZ'],"(C3)(-{0,1}(([A-Z][A-Z][A-Z])|([A-Z0-9]{1,4}))..."


In [10]:
#Do the same thing for the organizations
df_org = pd.read_csv("processed_itu_organizations.csv")
df_org.head()

res_org = []
for index, entry in df_org.iterrows():
    res_org.append(generate_expressions(entry['description'], ast.literal_eval(entry['callsign']), ast.literal_eval(entry['suffix'])))
    
#Append To Dataframe
df_org['regex'] = res_org

#Save Dataframe To New CSV
df_org.to_csv('processed_itu_organizations_regex.csv', index=False)

In [61]:
df_org.head()

,name,description,callsign,suffix,regex
0,International Civil Aviation Organization,general,['4Y'],['AAA-ZZZ'],"(4Y)(-{0,1}(([A-Z][A-Z][A-Z])|([A-Z0-9]{1,4}))..."
1,United Nations,general,['4U'],['AAA-ZZZ'],"(4U)(-{0,1}(([A-Z][A-Z][A-Z])|([A-Z0-9]{1,4}))..."
2,World Meteorological Organization,general,['C7'],['AAA-ZZZ'],"(C7)(-{0,1}(([A-Z][A-Z][A-Z])|([A-Z0-9]{1,4}))..."


In [38]:
# How Many Unique Countries Are There?
#for i in l: print("- %s" % i)

#219 Unique Keys

#422 / 2 == 211 Countries
print("Entries:%s" % len(df))
print("Size Unique Countries:%s" % len(df['nation'].unique()))
print("Size Unique ISO Codes:%s" % len(df['iso codes'].unique()))

#What duplicates exist -For debugging purposes

unl = df['nation'].unique()
#print("codes")
uic = df['iso codes'].unique()
# for i in range(max(len(unl),len(uic))):
#     if i < len(unl) and i < len(uic):
#         print("Line Up = %s | %s" % (unl[i],uic[i]))
#     elif i < len(unl):
#         print("Remaining unl %s" % unl[i])
#     elif i < len(uic):
#         print("Remaining uic %s" % uic[i])

for code in df['callsign']:
    #print(ast.literal_eval(code)[0])
    if '-' in code:
        print(code)
    
# idx = pd.Index(df['iso codes'])
# count = 0
# for item in idx.duplicated():
#     if item:
#         print("Duplicate[%s] : iso=%s | nation=%s" % (count,df.iloc[count]['iso codes'],df.iloc[count]['nation']))
#     count += 1


Entries:408
Size Unique Countries:217
Size Unique ISO Codes:217
['VP-A']
['VP-B','VQ-B','VR-B']
['VP-L']
['VP-C']
['VP-F']
['OY-H']
['VP-G']
['OY-H']
['B-H', 'B-K', 'B-L']
['B-M']
['VP-M']
['SU-Y']
['F-OD']
['VQ-H']
['F-OH']
['VQ-T']


In [39]:
#CREATE THE API

import pandas as pd
import ast
import re

class ARP:
    
    dataset_files = {"countries":"processed_itu_countries_regex.csv",
                     "organizations":"processed_itu_organizations_regex.csv"}
    
    def __init__(self):
        self.datasets_loaded = False
        self.load_datasets(self.dataset_files)
            
    def load_datasets(self, files):
        try:
            if files is not None:
                self.dfs = {}
                for key, f in files.items():
                    self.dfs[key] = pd.read_csv(f)
                self.datasets_loaded = True
            else:
                self.print_dataset_error()        
        except:
            self.print_dataset_error()
            
    def parse(self, callsign): 
        res = []
        if not self.datasets_loaded:
            self.load_datasets(self.dataset_files)
        else:
            #parse the data
            for key, df in self.dfs.items():
                for index, entry in df.iterrows():
                    result = re.match(entry['regex'], callsign)
                    if result is not None:
                        #print("Success!")
                        #print(entry)
                        res.append({"nation":entry["nation"],"description":entry["description"],"iso codes":entry["iso codes"]})
                        
        return res
    
    def print_dataset_error(self):
        print("Can't identify dataset, try loading it manually")
        print("Use load_dataset(<tags.csv>)")
        

In [40]:
arp = ARP()
res = arp.parse("N1111")
print(res)

[{'nation': 'United States', 'description': 'general', 'iso codes': "['US', 'USA']"}]


In [41]:
# Testing How Well The Extractor Works!!!

# Tests were conducted on the FR24 Solr Collection located on 69.93
# Number of Ingested Records
# 7862289818
# Number of Unique Tail Numbers to Identify

# 25
df_tails = pd.read_csv("tail_test.csv")
print(df_tails.shape)
df_tails.head()

(254399, 2)


,tail,count
0,-D-0870,16793
1,.CWh,132
2,000,23787
3,00000000,3594692
4,002,7690


In [42]:
%%time

# Lets Evaluate the Parser
arp = ARP()

none = []
one = []
more_than_one_same_country = []
more_than_one_diff_country = []

for index, utail in df_tails.iterrows():
    res = arp.parse(utail["tail"])
    if len(res) == 0:
        none.append(utail["tail"])
    elif len(res) == 1:
        one.append((utail["tail"],res))
    else:
        #more than one... country?
        same = True
        c = res[0]["nation"]
        for r in range(len(res)-1):
            if res[r+1]["nation"] != c:
                same = False
                break
        if same:
            more_than_one_same_country.append((utail["tail"],res))
        else:
            more_than_one_diff_country.append((utail["tail"],res))

    if index % 1000 == 0 and index > 0:
        print("Completed %s tail numbers" % index)       
        print("None Size:%s" % len(none))
        print("One Size:%s" % len(one))
        print("More Than One (Same Country) Size:%s" % len(more_than_one_same_country))
        print("More Than One (Different Country) Size:%s" % len(more_than_one_diff_country))
        print("-------------------------")

Completed 1000 tail numbers
None Size:258
One Size:613
More Than One (Same Country) Size:0
More Than One (Different Country) Size:130
-------------------------
Completed 2000 tail numbers
None Size:456
One Size:1409
More Than One (Same Country) Size:6
More Than One (Different Country) Size:130
-------------------------
Completed 3000 tail numbers
None Size:567
One Size:2298
More Than One (Same Country) Size:6
More Than One (Different Country) Size:130
-------------------------
Completed 4000 tail numbers
None Size:607
One Size:2833
More Than One (Same Country) Size:24
More Than One (Different Country) Size:537
-------------------------
Completed 5000 tail numbers
None Size:607
One Size:3038
More Than One (Same Country) Size:24
More Than One (Different Country) Size:1332
-------------------------
Completed 6000 tail numbers
None Size:607
One Size:3064
More Than One (Same Country) Size:24
More Than One (Different Country) Size:2306
-------------------------
Completed 7000 tail numbers
No

Completed 51000 tail numbers
None Size:1189
One Size:14639
More Than One (Same Country) Size:28465
More Than One (Different Country) Size:6708
-------------------------
Completed 52000 tail numbers
None Size:1342
One Size:15486
More Than One (Same Country) Size:28465
More Than One (Different Country) Size:6708
-------------------------
Completed 53000 tail numbers
None Size:1359
One Size:16469
More Than One (Same Country) Size:28465
More Than One (Different Country) Size:6708
-------------------------
Completed 54000 tail numbers
None Size:1362
One Size:17466
More Than One (Same Country) Size:28465
More Than One (Different Country) Size:6708
-------------------------
Completed 55000 tail numbers
None Size:1664
One Size:17984
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6708
-------------------------
Completed 56000 tail numbers
None Size:2247
One Size:18401
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6708
------------

Completed 100000 tail numbers
None Size:2463
One Size:62045
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848
-------------------------
Completed 101000 tail numbers
None Size:2463
One Size:63045
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848
-------------------------
Completed 102000 tail numbers
None Size:2464
One Size:64044
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848
-------------------------
Completed 103000 tail numbers
None Size:2464
One Size:65044
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848
-------------------------
Completed 104000 tail numbers
None Size:2464
One Size:66044
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848
-------------------------
Completed 105000 tail numbers
None Size:2464
One Size:67044
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848
------

Completed 149000 tail numbers
None Size:2493
One Size:111015
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848
-------------------------
Completed 150000 tail numbers
None Size:2493
One Size:112015
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848
-------------------------
Completed 151000 tail numbers
None Size:2493
One Size:113015
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848
-------------------------
Completed 152000 tail numbers
None Size:2493
One Size:114015
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848
-------------------------
Completed 153000 tail numbers
None Size:2493
One Size:115015
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848
-------------------------
Completed 154000 tail numbers
None Size:2493
One Size:116015
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848


Completed 197000 tail numbers
None Size:2495
One Size:159013
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848
-------------------------
Completed 198000 tail numbers
None Size:2495
One Size:160013
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848
-------------------------
Completed 199000 tail numbers
None Size:2495
One Size:161013
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848
-------------------------
Completed 200000 tail numbers
None Size:2495
One Size:162013
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848
-------------------------
Completed 201000 tail numbers
None Size:2495
One Size:163013
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848
-------------------------
Completed 202000 tail numbers
None Size:2495
One Size:164013
More Than One (Same Country) Size:28645
More Than One (Different Country) Size:6848


Completed 245000 tail numbers
None Size:2865
One Size:204106
More Than One (Same Country) Size:31117
More Than One (Different Country) Size:6913
-------------------------
Completed 246000 tail numbers
None Size:2956
One Size:205005
More Than One (Same Country) Size:31127
More Than One (Different Country) Size:6913
-------------------------
Completed 247000 tail numbers
None Size:2956
One Size:206005
More Than One (Same Country) Size:31127
More Than One (Different Country) Size:6913
-------------------------
Completed 248000 tail numbers
None Size:2956
One Size:207005
More Than One (Same Country) Size:31127
More Than One (Different Country) Size:6913
-------------------------
Completed 249000 tail numbers
None Size:2956
One Size:208005
More Than One (Same Country) Size:31127
More Than One (Different Country) Size:6913
-------------------------
Completed 250000 tail numbers
None Size:2964
One Size:208997
More Than One (Same Country) Size:31127
More Than One (Different Country) Size:6913


In [43]:
for e_val in none:
    print(e_val)

-D-0870
.CWh
000
00000000
002
007
01024
015
0169
017
01AAQ
01BLUE
02-1098
02-BLUE
020
0207
0218
0223
023
0264
0265
0270
029
030
031
032
037
04-DW
040
041
042
043
047
048
04BLUE
04HR
050
052
05HS
072
073
0731
08-1201
08-8202
0828
08GH
09-311
0916
100008
102
102003
104
1041L
109
11008D
1101MD
11020F
1105
1121
11339
1170
1171
1172
12195
1223
1234
12345
13-002
13-AAT
13-UA
130343
135L-484
140111
140112
140116
14162
142804
1521
156111
1601
162144
163591
164407
167110
168764
169339
177
184JB
19-8316
19-8443
1931
196
1962
1AZC990
1DEX231
1DNT224
1DZA771
1DZD105
1DZD775
1DZD799
1EGA615
1EJF335
1EJF401
1EMB455
1EMT300
20-25
220665
24-5346
24-7502
24-7572
24-7734
24-7909
24-8003
24-8038
24-8042
24-8094
24-8118
24-8292
24-8317
24-8344
24-8379
24-8398
24-8655
24-8657
25-BLUE
29420a6
306
3085
31694
31BLACK
31E
321
324
32BLACK
33005
33006
33182
334
33ANI
33BLACK
343
345
3451
34BLACK
35-730
350
351
352
353
35314
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
3732X
374

In [45]:
for e_val in more_than_one_diff_country:
    print(e_val)

('2-AKOP', [{'nation': 'Guernsey', 'description': 'general', 'iso codes': "['GG', 'GGY']"}, {'nation': 'United Kingdom', 'description': 'general', 'iso codes': "['GB', 'GBR']"}])
('2-ANLD', [{'nation': 'Guernsey', 'description': 'general', 'iso codes': "['GG', 'GGY']"}, {'nation': 'United Kingdom', 'description': 'general', 'iso codes': "['GB', 'GBR']"}])
('2-ASIA', [{'nation': 'Guernsey', 'description': 'general', 'iso codes': "['GG', 'GGY']"}, {'nation': 'United Kingdom', 'description': 'general', 'iso codes': "['GB', 'GBR']"}])
('2-ATRC', [{'nation': 'Guernsey', 'description': 'general', 'iso codes': "['GG', 'GGY']"}, {'nation': 'United Kingdom', 'description': 'general', 'iso codes': "['GB', 'GBR']"}])
('2-ATRD', [{'nation': 'Guernsey', 'description': 'general', 'iso codes': "['GG', 'GGY']"}, {'nation': 'United Kingdom', 'description': 'general', 'iso codes': "['GB', 'GBR']"}])
('2-ATRE', [{'nation': 'Guernsey', 'description': 'general', 'iso codes': "['GG', 'GGY']"}, {'nation': 'U

('B-7635', [{'nation': 'China', 'description': 'general', 'iso codes': "['CN', 'CHN']"}, {'nation': 'Taiwan', 'description': 'general', 'iso codes': "['TW', 'TWN']"}])
('B-7636', [{'nation': 'China', 'description': 'general', 'iso codes': "['CN', 'CHN']"}, {'nation': 'Taiwan', 'description': 'general', 'iso codes': "['TW', 'TWN']"}])
('B-7637', [{'nation': 'China', 'description': 'general', 'iso codes': "['CN', 'CHN']"}, {'nation': 'Taiwan', 'description': 'general', 'iso codes': "['TW', 'TWN']"}])
('B-7638', [{'nation': 'China', 'description': 'general', 'iso codes': "['CN', 'CHN']"}, {'nation': 'Taiwan', 'description': 'general', 'iso codes': "['TW', 'TWN']"}])
('B-7639', [{'nation': 'China', 'description': 'general', 'iso codes': "['CN', 'CHN']"}, {'nation': 'Taiwan', 'description': 'general', 'iso codes': "['TW', 'TWN']"}])
('B-7667', [{'nation': 'China', 'description': 'general', 'iso codes': "['CN', 'CHN']"}, {'nation': 'Taiwan', 'description': 'general', 'iso codes': "['TW', 'TW

('HB-ZIB', [{'nation': 'Liechtenstein', 'description': 'general', 'iso codes': "['LI', 'LIE']"}, {'nation': 'Switzerland', 'description': 'general', 'iso codes': "['CH', 'CHE']"}])
('HB-ZIE', [{'nation': 'Liechtenstein', 'description': 'general', 'iso codes': "['LI', 'LIE']"}, {'nation': 'Switzerland', 'description': 'general', 'iso codes': "['CH', 'CHE']"}])
('HB-ZIH', [{'nation': 'Liechtenstein', 'description': 'general', 'iso codes': "['LI', 'LIE']"}, {'nation': 'Switzerland', 'description': 'general', 'iso codes': "['CH', 'CHE']"}])
('HB-ZIJ', [{'nation': 'Liechtenstein', 'description': 'general', 'iso codes': "['LI', 'LIE']"}, {'nation': 'Switzerland', 'description': 'general', 'iso codes': "['CH', 'CHE']"}])
('HB-ZIK', [{'nation': 'Liechtenstein', 'description': 'general', 'iso codes': "['LI', 'LIE']"}, {'nation': 'Switzerland', 'description': 'general', 'iso codes': "['CH', 'CHE']"}])
('HB-ZIL', [{'nation': 'Liechtenstein', 'description': 'general', 'iso codes': "['LI', 'LIE']"}

In [58]:
# Evaluation Metrics

bins = {}
k = 'nation'
for e in one:
    if e[1][0][k] not in bins:
        bins[e[1][0][k]] = 1
    else:
        bins[e[1][0][k]] += 1

for e in more_than_one_same_country:
    if e[1][0][k] not in bins:
        bins[e[1][0][k]] = 1
    else:
        bins[e[1][0][k]] += 1
        
print(len(bins))

173


In [59]:
for key, c in bins.items():
    print("Country=%s | Count=%s" % (key,c))

Country=Monaco | Count=19
Country=Mauritius | Count=15
Country=Equatorial Guinea | Count=5
Country=Swaziland or Eswatini | Count=2
Country=China | Count=21
Country=Azerbaijan | Count=42
Country=Georgia | Count=23
Country=Montenegro | Count=8
Country=Sri Lanka | Count=30
Country=Israel | Count=158
Country=Libya | Count=29
Country=Cyprus | Count=38
Country=Morocco | Count=92
Country=Tanzania | Count=28
Country=Nigeria | Count=70
Country=Madagascar | Count=8
Country=Mauritania | Count=4
Country=Niger | Count=2
Country=Togo | Count=2
Country=Samoa | Count=2
Country=Uganda | Count=8
Country=Kenya | Count=109
Country=Senegal | Count=11
Country=Indonesia | Count=665
Country=Yemen | Count=5
Country=Malawi | Count=4
Country=Algeria | Count=85
Country=Barbados | Count=2
Country=Maldives | Count=74
Country=Guyana | Count=17
Country=Croatia | Count=50
Country=Ghana | Count=10
Country=Malta | Count=100
Country=Zambia | Count=1
Country=Kuwait | Count=43
Country=Malaysia | Count=327
Country=Nepal | C

In [55]:
# How Many Unique Countries Are There?
#for i in l: print("- %s" % i)

#219 Unique Keys

#422 / 2 == 211 Countries
print("Entries:%s" % len(df))
print("Size Unique Countries:%s" % len(df['nation'].unique()))
print("Size Unique ISO Codes:%s" % len(df['iso codes'].unique()))

#What duplicates exist -For debugging purposes

unl = df['nation'].unique()
#print("codes")
uic = df['iso codes'].unique()
# for i in range(max(len(unl),len(uic))):
#     if i < len(unl) and i < len(uic):
#         print("Line Up = %s | %s" % (unl[i],uic[i]))
#     elif i < len(unl):
#         print("Remaining unl %s" % unl[i])
#     elif i < len(uic):
#         print("Remaining uic %s" % uic[i])

for code in df['callsign']:
    #print(ast.literal_eval(code)[0])
    if '-' in code:
        print(code)
    
# idx = pd.Index(df['iso codes'])
# count = 0
# for item in idx.duplicated():
#     if item:
#         print("Duplicate[%s] : iso=%s | nation=%s" % (count,df.iloc[count]['iso codes'],df.iloc[count]['nation']))
#     count += 1

Entries:408
Size Unique Countries:217
Size Unique ISO Codes:217
['VP-A']
['VP-B','VQ-B','VR-B']
['VP-L']
['VP-C']
['VP-F']
['OY-H']
['VP-G']
['OY-H']
['B-H', 'B-K', 'B-L']
['B-M']
['VP-M']
['SU-Y']
['F-OD']
['VQ-H']
['F-OH']
['VQ-T']


In [ ]:
#Longest Code (After Dash or Known Character Set)
df_list = df[['iso codes','callsign','suffix']]
m = 3
c = "['AF', 'AFG']"
k = "['T6', 'YA']"
s = "['AAA-ZZZ']"
for key, row in df_list.iterrows():
    for i in row['suffix'].split("-"):
        if len(i) > m:
            print("New Max")
            m = len(i)
            c = row['iso codes']
            k = row['callsign']
            s = row['suffix']
print("Final Max")
print(c, k, s)
# Max Length of Final is 5...
# Max Length of Beginning is 3...
    #print(row['iso codes'], row['callsign'], row['suffix'])